In [1]:
import test_environment as t
from test_environment import TestEnvironment
import yaml

ModuleNotFoundError: No module named 'openvino.runtime'

In [2]:
import yaml
import cv2
import numpy as np

with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.loader.FullLoader)
video = cv2.VideoCapture("images/NTHUDDD_dataset/Training_Evaluation_Dataset/Training Dataset/036/noglasses/nonsleepyCombination.avi")
valid, img = video.read()
for i in range(0, 395):
    valid, img = video.read()

modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
h, w = img.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0,
(300, 300), (104.0, 117.0, 123.0))
net.setInput(blob)
faces = net.forward()
#to draw faces on image
for i in range(faces.shape[2]):
    confidence = faces[0, 0, i, 2]
    if confidence > 0.5:
        box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
        (x, y, x1, y1) = box.astype("int")
        cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)

cv2.imshow("", img)
cv2.waitKey()

-1

In [5]:
def draw_annotation_box(img, rotation_vector, translation_vector, camera_matrix, color=(255, 255, 0), line_width=2):
    """Draw a 3D box as annotation of pose"""
    point_3d = []
    dist_coeffs = np.zeros((4,1))
    rear_size = 1
    rear_depth = 0
    point_3d.append((-rear_size, -rear_size, rear_depth))
    point_3d.append((-rear_size, rear_size, rear_depth))
    point_3d.append((rear_size, rear_size, rear_depth))
    point_3d.append((rear_size, -rear_size, rear_depth))
    point_3d.append((-rear_size, -rear_size, rear_depth))

    front_size = img.shape[1]
    front_depth = front_size*2
    point_3d.append((-front_size, -front_size, front_depth))
    point_3d.append((-front_size, front_size, front_depth))
    point_3d.append((front_size, front_size, front_depth))
    point_3d.append((front_size, -front_size, front_depth))
    point_3d.append((-front_size, -front_size, front_depth))
    point_3d = np.array(point_3d, dtype=np.float).reshape(-1, 3)

    # Map to 2d img points
    (point_2d, _) = cv2.projectPoints(point_3d,
                                      rotation_vector,
                                      translation_vector,
                                      camera_matrix,
                                      dist_coeffs)
    point_2d = np.int32(point_2d.reshape(-1, 2))
    

    # # Draw all the lines
    # cv2.polylines(img, [point_2d], True, color, line_width, cv2.LINE_AA)
    k = (point_2d[5] + point_2d[8])//2
    # cv2.line(img, tuple(point_2d[1]), tuple(
    #     point_2d[6]), color, line_width, cv2.LINE_AA)
    # cv2.line(img, tuple(point_2d[2]), tuple(
    #     point_2d[7]), color, line_width, cv2.LINE_AA)
    # cv2.line(img, tuple(point_2d[3]), tuple(
    #     point_2d[8]), color, line_width, cv2.LINE_AA)
    
    return(point_2d[2], k)

In [ ]:
import cv2
import numpy as np
import region_detection as roi
import math

with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.loader.FullLoader)
video = cv2.VideoCapture("images/adrian/head_pose.mp4")
# video = cv2.VideoCapture("images/NTHUDDD_dataset/Training_Evaluation_Dataset/Training Dataset/036/noglasses/nonsleepyCombination.avi")
valid, img = video.read()
# for i in range(0, 392):
#     valid, img = video.read()
while valid:
    size = img.shape
    faces, frame = roi.mediapipe_face_mesh(img, debug=False)
    
    print("ENTER")
    if faces is None or faces.multi_face_landmarks is None:
        valid, img = video.read()
        continue
    print("NOT GOING")

    landmarks = faces.multi_face_landmarks[0]
    
    points_2D_indexes = [1, 152, 263, 33, 291, 61]
    points_2D = np.array([(landmarks.landmark[index].x * size[1], landmarks.landmark[index].y * size[0]) for index in points_2D_indexes])
    points_3D = np.array([(landmarks.landmark[index].x * size[1], landmarks.landmark[index].y * size[0], landmarks.landmark[index].z) for index in points_2D_indexes])

    dist_coeffs = np.zeros((4,1))  
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                            [[focal_length, 0, center[0]],
                            [0, focal_length, center[1]],
                            [0, 0, 1]], dtype = "double"
                            )

    points_3D = np.array([
                (0.0, 0.0, 0.0),       #Nose ti
                (0.0, -330.0, -65.0),  #Chin
                (-225.0, 170.0, -135.0),#Left eye corner
                (225.0, 170.0, -135.0), #Right eye corner 
                (-150.0, -150.0, -125.0),#Left mouth 
                (150.0, -150.0, -125.0) #Right mouth 
                ])     
    success, rotation_vector, translation_vector = cv2.solvePnP(points_3D, points_2D, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
    nose_end_point2D, jacobian = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    for p in points_2D:
        cv2.circle(img, (int(p[0]), int(p[1])), 3, (0,0,255), -1)

    p1 = ( int(points_2D[0][0]), int(points_2D[0][1]))
    
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
    x1, x2 = draw_annotation_box(img, rotation_vector, translation_vector, camera_matrix)
    
    cv2.line(img, p1, p2, (255,255,255), 2)
    #cv2.line(img, tuple(x1), tuple(x2), (255, 255, 0), 2)
    try:
        m = (p2[1] - p1[1])/(p2[0] - p1[0])
        ang1 = int(math.degrees(math.atan(m)))

        x = p2[0] - p1[0]
        y = p2[1] - p1[1]
        z = 1 - 0 # asumimos que p1[z] = 0
        pitch = int(math.degrees(math.acos(y / math.sqrt(x**2 + y**2 + z**2)))) - 90  # shiftamos 90 grados para que recto = 0
        yaw = int(math.degrees(math.acos(x / math.sqrt(x**2 + y**2 + z**2))))  # shiftamos 90 grados para que recto = 0
    except:
        ang1 = 90
        
    try:
        m = (x2[1] - x1[1])/(x2[0] - x1[0])
        x = x2[0] - x1[0]
        y = x2[1] - x1[1]
        z = 1 - 0 # asumimos que p1[z] = 0
        ang2 = int(math.degrees(math.atan(-1/m)))
        yaw = int(math.degrees(math.acos(x / math.sqrt(x**2 + y**2 + z**2)))) - 90 # shiftamos 90 grados para que recto = 0
    except:
        ang2 = 90
    # Display image
    font = cv2.FONT_HERSHEY_SIMPLEX 
    cv2.putText(img, str(ang1), tuple(p1), font, 2, (128, 255, 255), 3)
    cv2.putText(img, str(pitch), tuple((100, 100)), font, 2, (0, 255, 0), 3)
    cv2.putText(img, str(yaw), tuple((100, 200)), font, 2, (0, 255, 0), 3)
    cv2.putText(img, str(ang2), tuple((100, 500)), font, 2, (255, 255, 128), 3)
    cv2.imshow('', img)
    cv2.waitKey()
    valid, img = video.read()
cv2.waitKey()

In [6]:
import cv2


gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
equalized = clahe.apply(gray_image)

cv2.imshow("", image)
cv2.imshow("a", gray_image)
cv2.imshow("a2", equalized)
cv2.waitKey()

frame_metrics, image, _ = mo.process_frame(image, config["metric_obtention"])
cv2.imshow(' ', image)
cv2.waitKey()

NameError: name 'mo' is not defined

: 

In [2]:
test_env = TestEnvironment()

In [3]:
with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.loader.FullLoader)

In [4]:
videos_and_labels = (test_env.get_videos_and_labels_NTHUDDD())

In [5]:
import metrics_obtention as mo
df_list = mo.create_dataset_from_videos_NTHU(videos_and_labels, target_folder="NTHUDDD_dataset/", config=config["metric_obtention"])

[]


ValueError: Length of values (0) does not match length of index (3330)